In the scope of the first assignment of the Quantitative Case Study course, a classification problem was presented in the form of a  Kaggle competition.<br>
The problem involves the published data containing information on a range of products from different establishments. The purpose of this work is to use the provided data to create a model, which is able to correctly classify if a product is being sold with a discount or not.<br>
Before starting, it should be said, that usually there is only one explanation per subsection. This explanation can be at the start or at the end in a separte subsection of that group.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore

warnings.filterwarnings("ignore")                     
%matplotlib inline
sns.set(style="white", color_codes=True)
pd.set_option('display.max_columns', None)

The first step of development of this project was to import the necessary basic libraries other functions to enabled an easier analysis of the data, such as: 
* filtering all the warning that were not necessary
* automatically ploting the graphs
* set a basic style for seaborn that can be overwritten later
* force jupyter to show all the columns from any data frame.

In [ ]:
target=pd.read_csv('travel_test.csv') #change file
df=pd.read_csv('travel_train.csv') #change file
numbers=['MONTANTE']
integers=[]
continuous=['MONTANTE']
categorical=['CATEGORIA','DESCRICAO CATEGORIA', 'DATA']
X=categorical+numbers
Y = 'target'
#df.loc[:,Y]=df.loc[:,Y].astype('category')

Secondly, both the trainning and test data sets were loaded and their attributes separated according to their variable type. <br>
The "d " variables classified as categorical variables and the "x" variables, classified as numerical, were further separated between discrete (x1, x2, x4, x6, x7 and x8) and continuous (x3, x5 and x9) variables to facilitate the analysis of the data set.

# Analysis

Afterwards, a preliminary analysis was conducted starting by analysing the size of the data set, which is composed by 28 variables and 3676 entries. The variables presented are the "d" and "x" varaibles previously mentioned, the target variable "y" and the identificator "i". <br>
The imbalance of the data set was verified by the frequencies of the target variable, which is composed by 2920 "0" and 756 "1". <br>
 Finalizing this overview it is possible to infere from the tables describing the numeric and categorical atributes that the variable "x9" has missing values by having a lower count than the remaining numeric variables and all the categorical variables are binary with very high frequency for their mode "0" compared to the value "1".

In [ ]:
#top of the data (first lines)
df.head()

In [ ]:
#size of the data frame
df.shape

In [ ]:
df[Y].value_counts()

In [ ]:
df.info()

In [ ]:
df[numbers].describe()

In [ ]:
# alterar variáveis
tmp=df.copy(deep=False)
tmp=tmp.loc[:,[Y]+categorical].astype('category')
tmp.describe(include='category')

# Feature engeneering

In [ ]:
df["DATA"] = pd.to_datetime(df["DATA"])

In [ ]:
df['Month'] = df['DATA'].dt.strftime('%m')

In [ ]:
df

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit_transform(df['CATEGORIA'].astype(str) +"-"+ df["Month"].astype(str))
new_df=pd.DataFrame(lb.fit_transform(df['CATEGORIA'].astype(str) +"-"+ df["Month"].astype(str)))

In [ ]:
new_df

In [ ]:
df1=pd.concat([df.loc[:,['ID','MONTANTE']], new_df], axis=1, ignore_index=True)

In [ ]:
df1

In [ ]:
for i in range(2,110):
    df1.iloc[:,i]=df1.iloc[:,i]*df1.iloc[:,1]

df1

In [ ]:
list(lb.classes_)

In [ ]:
df1=df1.rename(columns={0: "ID"})

In [ ]:
last_df=df1.groupby(by='ID').sum()

In [ ]:
for i in range(2,110):
    last_df = last_df.rename({i: list(lb.classes_)[i-2]}, axis='columns')

In [ ]:
last_df = last_df.rename({1: 'Montante'}, axis='columns')

In [ ]:
last_df

In [ ]:
target_df=df.groupby(by='ID')['target'].sum()
target_df=pd.DataFrame(target_df)

In [ ]:
target_df

In [ ]:
target_df['target'].where(target_df['target'] == 0, 1, inplace=True)

In [ ]:
target_df

In [ ]:
df2=pd.concat([last_df, target_df], axis=1, ignore_index=True)

In [ ]:
for i in range(1,109):
    df2 = df2.rename({i: list(lb.classes_)[i-1]}, axis='columns')

In [ ]:
df2 = df2.rename({0: 'Montante'}, axis='columns')
df2 = df2.rename({109: 'target'}, axis='columns')

In [ ]:
df2

In [ ]:
df2["PesoLazerViagem"] = ((df2["17-01"] + df2["17-02"] + df2["17-03"] + df2["17-04"])/(df2["21-01"] + df2["21-02"] + df2["21-03"] + df2["21-04"] + df2["22-01"] + df2["22-02"] + df2["22-03"] + df2["22-04"])).fillna(0)

In [ ]:
df2

# Graphic analysis

In the variable x3, most values fall between 0 and 1, having the majority of the observations lying between 0 and 0.5, with peak in the range of 0.25-0.5. Also, there are some observations that are potential outliers, which can be spotted by looking at the bars of the ranges [1.75, 2[ and [2.75, 3[.

With regards to x5, the distribution of this variable is skewed to the right. Most of the observations lie below the value 3.1. The tail of this distribution have more observations than the right tail of x3, which will result in a boxplot with more observations that lie beyond the upper fence than the boxplot of the variable x3.

Finally, x9 has 3 bins that can be seen as local peaks. The interval of these bins are [1.00, 3.65[, [16.90, 19.55[ and [40.75, 4.40[. The histogram of this variable contrasts with the histograms of x3 and x5, that clearly have right tails with progressively less observations as the value on the x-axis increases. In this variable, there are many observations from the minimum to the maximum value observed in it, which could mean that this variable has no outliers (to be confirmed looking at the boxplot of this variable).

The observations of the variable x1 are balanced, having close numbers of observations with each of the 7 existing values, unlike the other variables in these plots. X4 has the vast majority of the observations in one value only (out of 3) and x2 is unbalanced, having an uneven distribution of observations among the 4 different observed values. The variables x6, x7 and x8 are extremely unbalanced, with many observations concentrated in only a few number of observed values, and a very small number of observations in many observed values.

The boxplots of the variables x1, x2 and x9, it can be seen that there are no outliers and the distance of the 1st quartile and the 3rd quartile to the median is similar in each of these variables, which hints that it's very likely that the median and the mean of these variables are close (in relative terms), i.e., the distribution skewness of these variables may be be very low.

Looking at the boxplots of the variables x3, x5, x7 and x8 show many outliers and that the distance of the median to the 1st and to the 3rd quartile aren't similar, which means that the distributions of these variables are skewed. Apart from x7, the other 3 variables have many outliers.

The boxplot of the variable x4 shows that the median, 1st and 3rd quartile are equal. Since this variable is discrete, it means that there are only 3 different values observed in this variable (1, 2 and 3) and that at least 75% of the observations are equal to 1, which makes all the observations equal to 2 and 3 outliers.

With regards to the variable x6, the only meaningful information that can be drawn is that it has no outliers.

All the categorical variables are extremely unbalanced. Each of them have only 2 categories, and the vast majority of the observations fall into only one category of each variable. 

The conclusion drawn from the pie chars is the same as the one drawn from the barplots, with the additional information that at least 83% of the observations fall into only one category of each variable.

With regards to the numerical variables, only 5 variables have outliers (x3, x4, x5, x7 and x8). All the outliers observed are upper outliers. Also, the variables x3 and x5 many outliers, whereas x4, x7 and x8 have much less.

It is possible to confirm that among all variables, only x9 has missing values.

It was also verified that there are no duplicate rows in the training set.

In [ ]:
# histograms

for i in continuous:
    fig, ax = plt.subplots(1,1, figsize=(15, 6))
    #plots histogram, returns counts, bin border values, and the bars themselves
    h_vals, h_bins, h_bars = ax.hist(df[f'{i}'], bins=100, edgecolor="white")
    #plot x ticks at the place where the bin borders are
    ax.set_xticks(np.round_(h_bins, decimals=2))
    ax.set_title(f'{i}')
    plt.show()

In [ ]:
# barplots for discrete variables

for i in integers:
    fig, ax = plt.subplots(1,1, figsize=(15, 6))
    sns.countplot(y = df[i][1:],data=df, order=df[i][1:].value_counts().index, palette='Blues_r')
    fig.text(0.1, 0.95, f'{i}', fontsize=16, fontweight='bold', fontfamily='serif')
    plt.xlabel(' ', fontsize=20)
    plt.ylabel('')
    plt.yticks(fontsize=13)
    plt.box(False)

In [ ]:
# boxplots

for i in numbers:
    fig, ax = plt.subplots(1,1, figsize=(5, 6))
    sns.boxplot(y=df[i])
    fig.text(0.1, 0.95, f'{i}', fontsize=16, fontweight='bold', fontfamily='serif')
    plt.ylabel('')
    frame1 = plt.gca()
    frame1.axes.get_xaxis().set_visible(False)
    frame1.axes.get_yaxis().set_visible(False)
    plt.yticks(fontsize=13)
    plt.box(False)

In [ ]:
# barplots for categorical variables

for i in categorical:
    fig, ax = plt.subplots(1,1, figsize=(15, 6))
    sns.countplot(y = df[i][1:],data=df, order=df[i][1:].value_counts().index, palette='Blues_r')
    fig.text(0.1, 0.95, f'{i}', fontsize=16, fontweight='bold', fontfamily='serif')
    plt.xlabel(' ', fontsize=20)
    plt.ylabel('')
    plt.yticks(fontsize=13)
    plt.box(False)

In [ ]:
#Pie chart

for i in categorical:
    fig, ax = plt.subplots(1,1, figsize=(15, 6))
    plt.pie(df[i].value_counts(), labels=df[i].unique(), autopct='%.0f%%')
    fig.text(0.1, 0.95, f'{i}', fontsize=16, fontweight='bold', fontfamily='serif')
    plt.xlabel(' ', fontsize=20)
    plt.ylabel('')
    plt.yticks(fontsize=13)
    plt.box(False)

Number of outliers:

In [ ]:
# check outliers
for i in numbers:
    var_q1 = df[i].quantile(0.25)
    var_q3 = df[i].quantile(0.75)
    outlier_top_lim = var_q3 + 1.5 * (var_q3 - var_q1)
    outlier_bottom_lim = var_q1 - 1.5 * (var_q3 - var_q1)
    print(i)
    print("Upper outliers: {}".format(df.loc[df[i] > outlier_top_lim, i].count()))
    print("Lower outliers: {}".format(df.loc[df[i] < outlier_bottom_lim, i].count()))

In [ ]:
from scipy import stats

for i in numbers:
    z_score_upper_outlier = df[stats.zscore(df[i]) > 3]
    z_score_lower_outlier = df[stats.zscore(df[i]) < -3]
    print(i)
    print("Upper outliers: {}".format(z_score_upper_outlier[i].count()))
    print("Lower outliers: {}".format(z_score_lower_outlier[i].count()))

In [ ]:
#check missing values

missing = []
for coln in df.columns:
    if df[coln].isnull().values.any():
        missing.append(coln)

missing

In [ ]:
# number of missing values
for i in missing:
    print(i + ': '+ str(df[i].isna().sum()))

In [ ]:
# duplicate rows

print(df.duplicated().sum())

# Multivariate analysis

To perform a multivariate analysis we started by comparing the values of the variables for the different values of the target variable, obtaining the pair plot. <br>
In the pair plot it is possible to analyse the distribution of a single numeric variable throught the density plots, as well as the relationship between two variables in the scatterplots. In this graphics, the orange represents y=1 and the blue y=0. In the scatter plots the y=0 and y=1 are not easily separated and in the density plots it is possible to, once again, verify the imbalance of the data, aswell as verify that the distributions for y=0 and y=1 are similar.The similarity in the distributions can also be seen in the boxplots of the numeric variables and the imbalance of the data set can be easily identified on the bar plots of the categoric variables. The boxplots are also able to represent the outliers found in the variables, which are evaluated and treated in the following chapters. <br>
Afterwards, a correlation analysis between the variables was performed. The only notable correlations found were between x4 and d1 with 0.74 and d10 and d13 with 0.62. This is information is valuable to perform feature selection, but as it will be presented in the Feature Selection chapter of the project, the feature selection process was automated. <br>


In [ ]:
# matrix graph

sns.pairplot(df[[Y]+numbers], hue=Y, size=3, diag_kind="kde")

In [ ]:
# correlation heatmap (only number variables)

corr = df[numbers].corr()
f, ax = plt.subplots(figsize=(12, 9))
ax = sns.heatmap(corr, cmap="Blues", annot=True)

In [ ]:
# correlation heatmap 

corr = df[[Y]+X].corr(method='spearman')
f, ax = plt.subplots(figsize=(24, 18))
ax = sns.heatmap(corr, cmap="Blues", annot=True)

In [ ]:
# barplots for categorical variables

for i in categorical:
    fig, ax = plt.subplots(1,1, figsize=(15, 6))
    sns.countplot(y = df[i][1:], hue=Y, data=df, order=df[i][1:].value_counts().index, palette='Blues_r')
    fig.text(0.1, 0.95, f'{i}', fontsize=16, fontweight='bold', fontfamily='serif')
    plt.xlabel(' ', fontsize=20)
    plt.ylabel('')
    plt.yticks(fontsize=13)
    plt.box(False)

In [ ]:
#boxplots

df[[Y]+numbers].boxplot(by=Y, figsize=(12, 6))

# Missing Values

Due to the limitations of certain models, it is sometimes necessary to substitute the missing values. There are some exceptions, that is, models that have an internal mechanism that can handle missing values. So, since we want to try different models, different methods were used to replace the missing values, in order to test which method is the best in our specific case.<br>

Firstly, confirmation of the exitence of missing values in the x9 variable was obtained. Afterwards, to make the df_predict_na a basic optuna automl model was used to find the best parameters of a catboostregressor, to predict the missing values of the x9 variable.

In [ ]:
def df_mean(df):
    for i in missing:
        df[i]=df[i].fillna(df[i].mean())
    return df

In [ ]:
def df_median(df):
    for i in missing:
        df[i]=df[i].fillna(df[i].median())
    return df

In [ ]:
def df_value(df):
    for i in missing:
        df[i]=df[i].fillna(9999)
    return df

In [ ]:
def df_remove_row(df):
    df = df.dropna()
    return df

In [ ]:
def df_remove_col(df):
    df = df.dropna(axis=1)
    return df

In [ ]:
def df_ffill(df):
    for i in missing:
        df[i] = df[i].fillna(method='ffill')
    return df

In [ ]:
def df_bfill(df):
    for i in missing:
        df[i] = df[i].fillna(method='bfill')
    return df

In [ ]:
from catboost import CatBoostRegressor
import optuna
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def objective(trial):
    
    # Definition of space search
    depth = trial.suggest_int('depth', 3, 16)
    learning_rate = trial.suggest_float('learning_rate', 0, 1, step=0.05)
    iterations = trial.suggest_int('iterations', 2, 2000)

    # Classifier definition
    model = CatBoostRegressor(depth=depth,
                                learning_rate=learning_rate,
                                iterations=iterations,logging_level='Silent')
    y_pred=model.fit(df[X].fillna(df['x9'].mean()),df[Y]).predict(df[X].fillna(df['x9'].mean()))

    return r2_score(df[Y],y_pred)

study=optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

In [ ]:
def df_predict_na(df):
    from sklearn.linear_model import ElasticNet
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import Ridge
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.linear_model import SGDRegressor
    from catboost import CatBoostRegressor
    from sklearn.kernel_ridge import KernelRidge
    from xgboost.sklearn import XGBRegressor
    from lightgbm import LGBMRegressor
    from  sklearn.preprocessing import StandardScaler
    from sklearn.metrics import r2_score

    scaler = StandardScaler()
    df_replica=df.dropna().copy(deep=False)
    df_replica.index=list(range(len(df_replica)))

    standardized_df = pd.DataFrame(scaler.fit_transform(df_replica.loc[:,'x1':'x8']))

    mydf=df_replica.loc[:,categorical]

    for coln in standardized_df.columns:
        mydf[coln]=standardized_df[coln]

    y_train = df["x9"].dropna()
    
    X_train = mydf
    X_train.rename(columns={0:'x1',1:'x2',2:'x3', 3:'x4',4:'x5',5:'x6',6:'x7',7:'x8'},inplace=True)
    X_test = df.loc[:,'d1':'x8']
    
    model = CatBoostRegressor(depth= 5, learning_rate= 0.55, iterations= 636,logging_level='Silent')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(r2_score(y_train,model.predict(X_train)))
    print(plt.scatter(y=y_pred,x=range(len(np.array(y_pred)))))

    for i in range(0,len(df['x9'])):

        if pd.isnull(df.iloc[i,len(df.columns)-1]):
            df.iloc[i,len(df.columns)-1]=pd.DataFrame(y_pred).iloc[i,0]

    return df

In [ ]:
def df_knn(df):
    
    from sklearn.impute import KNNImputer

    imp = KNNImputer(n_neighbors=2, weights='uniform')
    idf=pd.DataFrame(imp.fit_transform(df[X]))
    idf.columns=df[X].columns
    idf.index=df[X].index
    train=idf

    return idf

# Balancing

Since this data set is very unbalanced, being the majority of y equal to 1, another step that could be used in the pre-processing phase is the the data set balancing. For this, we used two of the most common oversampling and undersampling methods used for this purpose.

In [ ]:
# Oversampling
def oversampling_random(df):
    from imblearn.over_sampling import RandomOverSampler
    from sklearn.model_selection import train_test_split

    over_sampler = RandomOverSampler(random_state=42)
    X_res, y_res = over_sampler.fit_resample(df[X], df[Y])

    return pd.concat([y_res, X_res], axis=1)

In [ ]:
# Oversampling
def oversampling_SMOTE(df):
    from imblearn.over_sampling import SMOTE

    over_sampler = SMOTE(k_neighbors=2)
    X_res, y_res = over_sampler.fit_resample(df[X], df[Y])

    return pd.concat([y_res, X_res], axis=1)

In [ ]:
#Undersampling
def undersampling_nearmiss(df):
    from imblearn.under_sampling import NearMiss
    
    under_sampler = NearMiss()
    X_res, y_res = under_sampler.fit_resample(df[X], df[Y])
    
    return pd.concat([y_res, X_res], axis=1)

In [ ]:
#Undersampling
def undersampling_random(df):
    from imblearn.under_sampling import RandomUnderSampler

    under_sampler = RandomUnderSampler(random_state=42)
    X_res, y_res = under_sampler.fit_resample(df[X], df[Y])
    return pd.concat([y_res, X_res], axis=1)

# Scaling Data

Although this step of the model was not real necessary for our final model, some of the models tested used this step to improve their results, even though, none of them is represented here. 

In [ ]:
def df_minmax(df):
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    idf=pd.DataFrame(scaler.fit_transform(df.loc[:,numbers]))
    idf.columns=df.loc[:,numbers].columns
    idf.index=df.loc[:,numbers].index
    return pd.concat([df[categorical],idf], axis=1)

In [ ]:
def df_standard(df):
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    idf=pd.DataFrame(scaler.fit_transform(df.loc[:,numbers]))
    idf.columns=df.loc[:,numbers].columns
    idf.index=df.loc[:,numbers].index
    return pd.concat([df[categorical],idf], axis=1)

# Outliers

As it was seen in the boxplots and in our analysis, there are some outliers in this data set. Sometimes it is not appropriate to keep these observations while training. So, it would be wise to try and do something with them in the pre-processing stage.<br>

Sadly, it was not possible to tune the lof and isolation forest, since we do not real know if an observation really is an outliers, thus meaning, we do not have a measure to base on while tunning the model. Therefore, random isolation forests and lofs were used. However, the IQR and z-score do not require tunning, so no random model regarding these were used.

In [ ]:
def find_outliers_iforest(df):    
    from sklearn.ensemble import IsolationForest

    forest_df=df.loc[:, :]
    clf = IsolationForest(random_state=0).fit(df[X]).predict(df[X])
    forest_df=forest_df[clf==1]
    return forest_df


In [ ]:
def find_outliers_IQR(df):
    df_IQR=df.loc[:, [Y, *categorical]]
    Q1=df[numbers].quantile(0.25)
    Q3=df[numbers].quantile(0.75)
    outlier_top_lim = Q3 + 1.5 * (Q3 - Q1)
    outlier_bottom_lim = Q1 - 1.5 * (Q3 - Q1)
    df_IQR[numbers]=df[numbers][~((df[numbers]<(outlier_bottom_lim)) | (df[numbers]>(outlier_top_lim)))]
    df_IQR=df_IQR.dropna()
    return df_IQR


In [ ]:
def find_outliers_z(df):
    from scipy import stats
    
    df_zscore=pd.DataFrame()
    for i in numbers:
        z = np.abs(stats.zscore(df[i]))
        idx_outliers = np.where(z>3, True, False)
        df_zscore[i]=pd.DataFrame(idx_outliers)
    df_zscore_train=df[~df_zscore]
    df_zscore_train.loc[:, [Y, *categorical]]=df.loc[:, [Y, *categorical]]
    df_zscore_train=df_zscore_train.drop('i', axis=1)
    df_zscore_train=df_zscore_train.dropna()
    return df_zscore_train

In [ ]:
# Local Outlier Factor (LOF), being -1 outlier
def find_outliers_lof(df):
    from sklearn.neighbors import LocalOutlierFactor

    lof = LocalOutlierFactor(n_neighbors=2, metric='manhattan')
    lof_prediction = lof.fit_predict(df)
    df_lof=df
    df_lof=df[lof_prediction==1]
    return df_lof


In [ ]:
#find out best parameter for DBSCAN

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
from matplotlib.pyplot import figure
from sklearn.cluster import DBSCAN

scaler = StandardScaler()
scaled_features = scaler.fit_transform(final_df) #substitute missing value here

# If your data has more than 2 dimensions, choose MinPts = 2*dim, where dim= the dimensions of your data set (Sander et al., 1998)
neigh = NearestNeighbors(n_neighbors=52)
nbrs = neigh.fit(scaled_features)
distances, indices = nbrs.kneighbors(scaled_features)
distances = np.sort(distances, axis=0)
distances = distances[:,1]

#The optimal value for epsilon will be found at the point of maximum curvature.
figure(figsize=(13, 6), dpi=80)
plt.style.use("fivethirtyeight")
plt.plot(range(len(distances)), distances)
plt.xticks(range(len(distances)))
plt.xlabel("Number of Clusters")
plt.ylabel("distances")
plt.show()

In [ ]:
def find_outliers_dbscan(X, eps_range=np.arange(5.0, 50.0, 0.5), min_samples_range=range(5, 15)):
    best_score = -1
    best_params = None
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(X)

    for eps in eps_range:
        for min_samples in min_samples_range:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            dbscan.fit(scaled_features)
            labels = dbscan.labels_

            # compute silhouette score for the current DBSCAN parameters
            if len(np.unique(labels)) > 1:
                score = silhouette_score(scaled_features, labels)

            # if the current score is better than the previous best score, update the best score and best parameters
            if score > best_score:
                best_score = score
                best_params = {'eps': eps, 'min_samples': min_samples}

    dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
    dbscan.fit(scaled_features)
    labels = dbscan.labels_
    df_dbscan=df2[labels!=-1]
    return df_dbscan

In [ ]:
df3=find_outliers_dbscan(df2)

In [ ]:
df3

# Feature Selection

Another important step of data mining is feature selection. Here we pick the atributes that are most important to make the predictions, that is, help the model get the highest accuracy possible.<br>

For this we used recursive feature selection with cross validation and a voting mechanism that used three feature selection models and then verified which features were in common in the mechanisms to choose the best features. This last mechanism was formed because the feature selection of these three models individualy had poor results. <br>

The method that gave the best results was the rfecv. In it, we used a randomforestclassifier, as it normally returned a similar data set when compared with the boosting models, but this one had a slight better performance (less run time) than the others. <br>

To make a part of the feature selection (exclude x4), we also made use of the explainability that is explored in the last chapter of the project.

In [ ]:
def rfecv_df(df):
    from sklearn.feature_selection import RFECV
    from sklearn.model_selection import StratifiedKFold
    from sklearn.ensemble import RandomForestClassifier

    rfecv = RFECV(estimator=RandomForestClassifier(), step=1, cv=10,scoring = 'f1_macro')
    rfecv=rfecv.fit(df.loc[:,'Montante':'9-04'], df.loc[:,'target'])
    new_df=df.loc[:,'Montante':'9-04':].loc[:,rfecv.support_]
    return new_df


In [ ]:
def voting_df(df, n):
    
    from sklearn.ensemble import RandomForestClassifier
    
    # ExtraTreesClassifier feature selection
    from sklearn.ensemble import ExtraTreesClassifier
    model = ExtraTreesClassifier(n_estimators = n)
    model.fit(df[X], df[Y])

    extra_trees=sorted(list(zip(model.feature_importances_,df[X].columns)), key =lambda a: a[0])[::-1][:n]
    x=[i[1] for i in extra_trees]
    
    # recursive feature selection
    from sklearn.feature_selection import RFE
    rfe = RFE(RandomForestClassifier(), n_features_to_select = n)
    fit = rfe.fit(df[X], df[Y])
    recursive_selection=sorted(list(zip(fit.ranking_,df[X].columns)), key =lambda a: a[0])[:n]
    y=[i[1] for i in recursive_selection]
    
    # SelectKBest feature selection
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import chi2

    test=SelectKBest(score_func=chi2, k=n)
    fit = test.fit(df[X], df[Y])
    selectk=sorted(list(zip(fit.scores_,df[X].columns)), key= lambda a: a[0])[::-1][:n]
    z=[i[1] for i in selectk]
    var=[i for i in z if i in x and i in z]
    
    return df.loc[:,var]

# Prepare data set

In this section, a sort of pipeline was formed. In it we pick which function we want from the ones that we have seen so far and then progressively apply to the previously processed data frame. So, it is possible to experiment and observe the results. The steps done to get the best model in this stage were these (feature selection was performed with the commented line, but it sometimes gives a slightly different result, so we wrote down the necessary data frame). However, after applying the explainability models that are at the end of this notebook, it was noticed that some variables explained very little of the data. So, some attempts were made and the best single prediction made with a model was gotten with the features at the end (d2, d11 and all numerical variables with the exception of x4). The reason to do this is explained in the last part 

In [ ]:
#df=pd.read_csv('training.csv')
df_no_na = df2
df_no_outliers = df_no_na
df_balanced= df_no_outliers
df_scaled= df_balanced
final_df = rfecv_df(df_scaled)
# final_df = df_scaled[['d2','d1f1']+numbers]
#final_df=df_scaled
#final_df = df_scaled[['d2','d11']+numbers[:3]+numbers[4:]]

In [ ]:
missing = []
for coln in df.columns:
    if df[coln].isnull().values.any():
        missing.append(coln)

missing

In [ ]:
final_df

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

for i in [LogisticRegression(), DecisionTreeClassifier(), KNeighborsClassifier(), GaussianNB(), DecisionTreeClassifier(), RandomForestClassifier(), SVC(), MLPClassifier(), GradientBoostingClassifier(), CatBoostClassifier(logging_level='Silent'), XGBClassifier(), LGBMClassifier()]:
    print(str(i.__class__.__name__) + ': ' + str(cross_val_score(i, final_df, target_df, cv=10, scoring='f1_macro').mean()))


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(final_df, target_df, test_size=0.3, random_state=0)

# Plot the ROC curve for each classifier
plt.figure(figsize=(8,6))
for clf in [LogisticRegression(), DecisionTreeClassifier(), KNeighborsClassifier(), GaussianNB(), DecisionTreeClassifier(), RandomForestClassifier(), SVC(), MLPClassifier(), GradientBoostingClassifier(), CatBoostClassifier(logging_level='Silent'), XGBClassifier(), LGBMClassifier()]:
    clf.fit(X_train, y_train)
    y_proba = clf.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='{} (AUC = {:.2f})'.format(clf.__class__.__name__, roc_auc))

plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Multiple Classifiers')
plt.legend(loc="lower right")
plt.show()

# Models

### Random Forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier(n_jobs=-1)
criterion = ['gini', 'entropy', 'log_loss']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
min_samples_split = [int(x) for x in np.linspace(5, 100, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(5, 100, num = 10)]
max_features = ['auto', 'sqrt', 'log2'] #The number of features to consider when looking for the best split:
max_leaf_nodes = [int(x) for x in np.linspace(10, 110, num = 11)]
bootstrap = [True, False]
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# max_features
hyperparameters = dict(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features=max_features, max_leaf_nodes=max_leaf_nodes, bootstrap=bootstrap)
randomizedsearch = RandomizedSearchCV(randomforest, hyperparameters, n_iter=100, cv=10, verbose=0, n_jobs=-1)
best_model_randomforest = randomizedsearch.fit(final_df, df_no_outliers[Y])

In [ ]:
print(best_model_randomforest.best_params_)
print(best_model_randomforest.best_score_)

In [ ]:
# confusion matrix with sample
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)
matrix = confusion_matrix(y_test, best_model_randomforest.predict(X_test))
print(best_model_randomforest.score(X_test, y_test))
dataframe = pd.DataFrame(matrix, index=[0,1], columns=[0,1])
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")

In [ ]:
# roc curve with sample
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)

# Get predicted probabilities
target_probabilities = best_model_randomforest.predict_proba(X_test)[:,1]

# Create true and false positive rates
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, target_probabilities)

# plot ROC curve
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

### XGBoost

In [ ]:
# xgboosted
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

for _ in range(10):
    xgboosted = XGBClassifier()
    hyperparameters = {'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7,0.8,0.9,1,1.5,2,2.5,3],
                  'max_depth': [4,5,6,7,8,9,10,11,12],
                  'n_estimators': [500,700,1000,1200,1400,1500,1700, 1800,2000,3000],
                  'min_child_weight': [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
                  'colsample_bytree': [0.8, 0.9, 1.0]}

    randomizedsearch = RandomizedSearchCV(xgboosted, hyperparameters, cv=5, verbose=0, n_jobs=-1)
    best_model_xgboosted = randomizedsearch.fit(final_df, df_no_outliers[Y])
    
    print(best_model_xgboosted.best_params_)
    print(best_model_xgboosted.best_score_)

In [ ]:
print(best_model_xgboosted.best_params_)
print(best_model_xgboosted.best_score_)

In [ ]:
# confusion matrix with sample
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)
matrix = confusion_matrix(y_test, best_model_xgboosted.predict(X_test))
print(best_model_xgboosted.score(X_test, y_test))
dataframe = pd.DataFrame(matrix, index=[0,1], columns=[0,1])
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")

In [ ]:
# roc curve with sample
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)

# Get predicted probabilities
target_probabilities = best_model_xgboosted.predict_proba(X_test)[:,1]

# Create true and false positive rates
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, target_probabilities)

# plot ROC curve
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

## Catboost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier

catboosted = CatBoostClassifier(logging_level='Silent')
params = {'depth':[4,5,6,7,8],
          'iterations':[1000,1500,2000],
          'learning_rate':[0.18,0.2,0.22], 
          'l2_leaf_reg':[0.08,0.1, 0.12],
          'border_count':[200],
          'loss_function': ['CrossEntropy'],
          'random_strength': [8,9,10,11,12],
          'min_data_in_leaf': [25,27,30, 32,35],
          'best_model_min_trees': [3,4,5]}

randomizedsearch = RandomizedSearchCV(catboosted, params, scoring='f1', cv=10, error_score='raise' )
best_model_catboost = randomizedsearch.fit(final_df, df_no_outliers[Y])

In [ ]:
print(best_model_catboost.best_params_)
print(best_model_catboost.best_score_)

In [ ]:
# confusion matrix with sample
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)
matrix = confusion_matrix(y_test, best_model_catboost.predict(X_test))
print(best_model_catboost.score(X_test, y_test))
dataframe = pd.DataFrame(matrix, index=[0,1], columns=[0,1])
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")

In [ ]:
# roc curve with sample
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)

# Get predicted probabilities
target_probabilities = best_model_catboost.predict_proba(X_test)[:,1]

# Create true and false positive rates
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, target_probabilities)

# plot ROC curve
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

## Adaboost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

decisiontree = DecisionTreeClassifier()
svc = SVC(random_state=0)
logistic = LogisticRegression(random_state=0, n_jobs=-1)
naive_bayes = GaussianNB()
knn = KNeighborsClassifier()

adaboost = AdaBoostClassifier(random_state=0)
base_estimator = [decisiontree, randomforest, svc, knn, logistic, naive_bayes]
n_estimators = [10, 20, 50, 100, 150, 200]
learning_rate = [0.1, 0.5, 1, 1.5, 2, 3, 4, 5, 10]
algorithm = ['SAMME', 'SAMME.R']

hyperparameters = dict(base_estimator=base_estimator, n_estimators=n_estimators, learning_rate=learning_rate, algorithm=algorithm)
randomizedsearch = RandomizedSearchCV(adaboost, hyperparameters, n_iter=100, cv=10, verbose=0, n_jobs=-1, scoring='f1')
best_model_ada = randomizedsearch.fit(final_df, df_no_outliers[Y])

In [ ]:
print(best_model_ada.best_params_)
print(best_model_ada.best_score_)

In [ ]:
# confusion matrix with sample
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)
matrix = confusion_matrix(y_test, best_model_ada.predict(X_test))
print(best_model_ada.score(X_test, y_test))
dataframe = pd.DataFrame(matrix, index=[0,1], columns=[0,1])
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")

In [ ]:
# roc curve with sample
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)

# Get predicted probabilities
target_probabilities = best_model_ada.predict_proba(X_test)[:,1]

# Create true and false positive rates
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, target_probabilities)

# plot ROC curve
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

## Gradient Boosted

In [ ]:
# Gradient Boosted
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

boosted_tree=GradientBoostingClassifier()
hyperparameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.05, 0.5, 12),
    "min_samples_leaf": np.linspace(0.05, 0.5, 12),
    "max_depth":[2,3,4,5,6,7,8,9],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10, 30, 50, 70, 100, 150,200, 300, 400, 500]
    }

randomizedsearch = RandomizedSearchCV(boosted_tree, hyperparameters, n_iter=100, cv=10, verbose=0, n_jobs=-1, scoring='f1')
best_model_gradient = randomizedsearch.fit(final_df, df_no_outliers[Y])

In [ ]:
print(best_model_gradient.best_params_)
print(best_model_gradient.best_score_)

In [ ]:
# confusion matrix with sample
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)
matrix = confusion_matrix(y_test, best_model_gradient.predict(X_test))
print(best_model_gradient.score(X_test, y_test))
dataframe = pd.DataFrame(matrix, index=[0,1], columns=[0,1])
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")

In [ ]:
# roc curve with data
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

# Get predicted probabilities
target_probabilities = best_model_gradient.predict_proba(final_df)[:,1]

# Create true and false positive rates
false_positive_rate, true_positive_rate, threshold = roc_curve(df[Y], target_probabilities)

# plot ROC curve
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

## LGBM

In [ ]:
# LGBM
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

LGBM = LGBMClassifier()
gridParams = {
    'learning_rate': np.logspace(0,-9, num=30),
    'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'num_leaves': [int(x) for x in np.linspace(10, 110, num = 11)], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart','goss', 'rf'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'subsample' : np.logspace(0,-9, num=30),
    'reg_alpha' : [0.3,0.6,0.9,1,1.2, 1.5, 1.7, 2],
    'reg_lambda' : [0.3,0.6,0.9,1,1.2, 1.5, 1.7, 2],
    'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)]
    }

randomizedsearch = RandomizedSearchCV(LGBM, gridParams, cv=10, n_jobs=-1, scoring='f1')
best_model_lgbm = randomizedsearch.fit(final_df, df_no_outliers[Y])

In [ ]:
print(best_model_lgbm.best_params_)
print(best_model_lgbm.best_score_)

In [ ]:
# confusion matrix with sample
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.33, random_state=1)
matrix = confusion_matrix(y_test, best_model_lgbm.predict(X_test))
print(best_model_lgbm.score(X_test, y_test))
dataframe = pd.DataFrame(matrix, index=[0,1], columns=[0,1])
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")

In [ ]:
# roc curve with data
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

# Get predicted probabilities
target_probabilities = best_model_lgbm.predict_proba(final_df)[:,1]

# Create true and false positive rates
false_positive_rate, true_positive_rate, threshold = roc_curve(df[Y], target_probabilities)

# plot ROC curve
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

## Explanation of models used

After the preprocessing was done, many models were attempted, but only the best models where kept (accuracy > 95%), as they were all near the final score of 97.170% (this is not the final accuracy, this will be explained later). It seems like that tree related models, especially boosted models gave good results when we use this data set. For example, sector vector machines gave an accuracy of around 80% while training; k-nearest neighbors gave decent results while training (93-94%). However the accuracy on Kaggle went under the 90% threshold; the other models attempted were the decision tree with accuracies around 94% (not many tests were done with these). Logistic regression was also experimented with but returned disappointing results with accuracies below 90%. <br>

To perform the hyperparameterization in these models we opted to used gridsearch but, since there were too many parameters to optimize, we opted for the randomizedsearch. Since we used random search, which does not return the best method at the start, we ran this hyperparmeterization method a couple of times until we obtained our best model.<br>

In this part of the work, we used ROC curves and confusion matrixes to understand what were the problems with the models. In most cases it was overfitting, many attempts were done to mitigate this problem (limiting depth, l2_reg, and other parameters that are incorporated in the catboost), but none were successful.

# AutoML

Another attempt was to use AutoML to perform hyperparameterization but we could not get better results than we did with a randomized grid search.

In [ ]:
import optuna
from sklearn import model_selection
from sklearn.model_selection import train_test_split

def objective(trial):
    
    # Definition of space search
    loss_function = trial.suggest_categorical('loss_function', ['Logloss', 'CrossEntropy'])
    iterations = trial.suggest_int('iterations', 500, 3000)
    depth = trial.suggest_int('max_depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate',0.1,0.3,step=0.02)
    l2_leaf_reg = trial.suggest_float('l2_leaf_reg',0,0.3,step=0.02)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 20, 40)
    best_model_min_trees = trial.suggest_int('min_data_in_leaf', 2, 10)


    
    # Classifier definition
    model = CatBoostClassifier(loss_function=loss_function,
                                iterations=iterations,
                                depth=depth,
                                learning_rate=learning_rate,
                                l2_leaf_reg=l2_leaf_reg,
                                min_data_in_leaf=min_data_in_leaf,
                                best_model_min_trees=best_model_min_trees,
                                logging_level = 'Silent')

    score=model_selection.cross_val_score(model, final_df, target_df, n_jobs=-1, cv=10, error_score='raise', scoring='f1_macro' )
    accuracy=score.mean()
    return accuracy 

study=optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
print(study.best_trial)

In [ ]:
study.best_params

In [ ]:
import optuna
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    
    # Definition of space search
    randomforest = RandomForestClassifier(n_jobs=-1)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    max_depth = trial.suggest_int('max_depth', 2, 500)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 500)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 500)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']) #The number of features to consider when looking for the best split:
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 500)
    n_estimators = trial.suggest_int('max_depth', 200, 3000)

    # Classifier definition
    model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features=max_features, max_leaf_nodes=max_leaf_nodes)

    score=model_selection.cross_val_score(model, final_df, df[Y], n_jobs=-1, cv=5, scoring='f1')
    accuracy=score.mean()
    return accuracy 

study=optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

In [ ]:
print(study.best_trial)

In [ ]:
study.best_params

In [ ]:
import optuna
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

def objective(trial):
    
    # Definition of space search
    learning_rate = trial.suggest_float('learning_rate',0.1,1.0,step=0.05)
    n_estimators = trial.suggest_int('n_estimators', 200, 800)
    num_leaves = trial.suggest_int('num_leaves', 2, 100)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 100)
    boosting_type = trial.suggest_categorical('boosting_type', ['gbdt', 'dart','goss'])
    objective = trial.suggest_categorical('objective', ['binary'])
    subsample=trial.suggest_float('subsample',0.1,1.0,step=0.05)
    max_depth = trial.suggest_int('max_depth', 2, 100)
    reg_alpha = trial.suggest_float('reg_alpha',0.1,1.5,step=0.05)
    reg_lambda = trial.suggest_float('reg_lambda',0.1,1.5,step=0.05)

    # Classifier definition
    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, num_leaves=num_leaves, 
                           min_samples_leaf=min_samples_leaf, boosting_type=boosting_type, 
                           objective=objective, subsample=subsample, max_depth=max_depth,
                           reg_alpha=reg_alpha, reg_lambda=reg_lambda)

    score=model_selection.cross_val_score(model, final_df, df[Y], n_jobs=-1, cv=5,error_score='raise', scoring='f1')
    accuracy=score.mean()
    return accuracy 

study=optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
print(study.best_trial)

In [ ]:
study.best_params

In [ ]:
import optuna
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

def objective(trial):
    
    # Definition of space search
    randomforest = XGBClassifier()
    learning_rate = trial.suggest_float('learning_rate',0.1,1.0,step=0.05)
    n_estimators = trial.suggest_int('n_estimators', 200, 3000)
    max_depth = trial.suggest_int('max_depth', 2, 500)
    min_child_weight = trial.suggest_float('min_child_weight',0.1,1.0,step=0.05)
    colsample_bytree = trial.suggest_float('colsample_bytree',0.1,1.0,step=0.05)

    # Classifier definition
    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, 
                           min_child_weight=min_child_weight, colsample_bytree=colsample_bytree)

    score=model_selection.cross_val_score(model, final_df, df[Y], n_jobs=-1, cv=5,error_score='raise', scoring='f1')
    accuracy=score.mean()
    return accuracy 

study=optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

In [ ]:
print(study.best_trial)

In [ ]:
study.best_params

In [ ]:
import optuna
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier

def objective(trial):
    
    # Definition of space search
    randomforest = GradientBoostingClassifier()
    learning_rate = trial.suggest_float('learning_rate',0.1,1.0,step=0.05)
    loss = trial.suggest_categorical('loss', ['log_loss', 'deviance', 'exponential'])
    min_samples_split=trial.suggest_float('min_samples_split',0.05,1.0,step=0.05)
    min_samples_leaf=trial.suggest_float('min_samples_leaf',0.05,1.0,step=0.05)
    max_depth = trial.suggest_int('max_depth', 2, 50)
    max_features = trial.suggest_categorical('max_features',['auto',"log2","sqrt"])
    criterion = trial.suggest_categorical('criterion',["friedman_mse",  "squared_error", 'mse'])
    subsample=trial.suggest_float('subsample',0.05,1.0,step=0.05)
    n_estimators = trial.suggest_int('n_estimators', 200, 3000)

    # Classifier definition
    model = GradientBoostingClassifier(learning_rate=learning_rate, loss=loss, min_samples_split=min_samples_split, 
                           min_samples_leaf=min_samples_leaf, max_depth=max_depth, max_features=max_features,
                           criterion=criterion, subsample=subsample, n_estimators=n_estimators)

    score=model_selection.cross_val_score(model, final_df, target_df, n_jobs=-1, cv=5,error_score='raise',scoring='f1_macro')
    accuracy=score.mean()
    return accuracy 

study=optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

In [ ]:
print(study.best_trial)

In [ ]:
study.best_params

# Extra attempts

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.linear_model import LogisticRegression
import numpy as np


# Create a logistic regression model
model = GradientBoostingClassifier()


# Generate the training curve
train_sizes, train_scores, test_scores = learning_curve(model, final_df, target_df, cv=5, scoring='f1', n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 10))

# Plot the learning curve
plt.plot(train_sizes, np.mean(train_scores, axis=1), 'o-', color='r', label='Training score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), 'o-', color='g', label='Cross-validation score')
plt.xlabel('Training size')
plt.ylabel('Score')
plt.legend(loc='best')
plt.show()

## Voting

In [ ]:
# importing utility modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
 
# importing machine learning models for prediction
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
 
# importing voting classifier
from sklearn.ensemble import VotingClassifier
 
# Splitting between train data into training and validation dataset
X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size=0.3, random_state=0)

# initializing all the model objects with default parameters
model_1 = GradientBoostingClassifier()
model_2 = XGBClassifier()
model_3 = GaussianNB()
 
# Making the final model using voting classifier
best_model = VotingClassifier(
    estimators=[('cat', model_1), ('xgb', model_2), ('lgbm', model_3)], voting='soft')
 
# training all the model on the train dataset
best_model.fit(X_train, y_train)
 
# predicting the output on the test dataset
pred_final = best_model.predict(X_test)
 
# printing log loss between actual and predicted value
print(log_loss(y_test, pred_final))
print(f1_score(y_test, pred_final))

## Stacking

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators= 1600, min_samples_split= 15, min_samples_leaf= 15, max_leaf_nodes= 100, max_features= 'sqrt', max_depth= 100, criterion= 'entropy', bootstrap= False)),
    ('xgb', XGBClassifier(reg_lambda= 0, reg_alpha= 0.1, n_estimators= 1000, min_child_weight= 1, max_depth= 6, learning_rate= 0.4, gamma= 0, colsample_bytree= 0.6))]


clf = StackingClassifier(
    estimators=estimators, final_estimator=CatBoostClassifier(nan_mode='Max',random_strength= 10, min_data_in_leaf= 30, loss_function= 'CrossEntropy', learning_rate= 0.2 , l2_leaf_reg= 0.1, iterations= 1000, depth= 5, border_count= 200, best_model_min_trees= 3))


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
     final_df, df[Y], stratify=df[Y], random_state=42 )
clf.fit(X_train, y_train).score(X_test, y_test,scoring='f1')

## Bagging classifier

In [ ]:
#16
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

X_train, X_test, y_train, y_test = train_test_split(final_df, df[Y], test_size = 0.3)

estimator_range = range(10,20)

models = []
scores = []

for n_estimators in estimator_range:

    # Create bagging classifier
    clf = BaggingClassifier(base_estimator=CatBoostClassifier(logging_level='Silent',nan_mode='Max',random_strength= 10, min_data_in_leaf= 30, loss_function= 'CrossEntropy', learning_rate= 0.2 , l2_leaf_reg= 0.1, iterations= 1000, depth= 5, border_count= 200, best_model_min_trees= 3), n_estimators = n_estimators)

    # Fit the model
    clf.fit(X_train, y_train)

    # Append the model and score to their respective list
    models.append(clf)
    scores.append(cross_val_score(clf, final_df, df[Y], scoring='f1').mean())

# Generate the plot of scores against number of estimators
plt.figure(figsize=(9,6))
plt.plot(estimator_range, scores)

# Adjust labels and font (to make visable)
plt.xlabel("n_estimators", fontsize = 18)
plt.ylabel("score", fontsize = 18)
plt.tick_params(labelsize = 16)

# Visualize plot
plt.show()

In [ ]:
from sklearn.ensemble import BaggingClassifier
from catboost import CatBoostClassifier

# Create bagging classifier
clf = BaggingClassifier(base_estimator=CatBoostClassifier(logging_level='Silent',nan_mode='Max',random_strength= 10, min_data_in_leaf= 30, loss_function= 'CrossEntropy', learning_rate= 0.2 , l2_leaf_reg= 0.1, iterations= 1000, depth= 5, border_count= 200, best_model_min_trees= 3), n_estimators = 11)

# Fit the model
clf.fit(final_df, df[Y])

## Extra models explanation

To mitigate overfitting, bagging, voting, and stacking were performed, but every attempt only worsened our final accuracy, even in training, so none were used in the final prediction.

# Predictions

In this part of the notebook the predictions were done. Here, we also added our best model, which did not need much manual preprocessing, needing only feature selection.

It seems like catboost does not require missing value substition because this model has an internal mechanism to do so, which substitutes the missing values by a value lower or higher than any other value. In this data set, there was no distinction in accuracy between both methods.

However, for simplicity's sake, as we said previously, in the next steps, we used the function df_value which substituted the missing values by 9999 and had the same accuracy.

Another thing that we discovered was that by changing the threshold of the predictions, we could improved the accuracy of the model to 0.97252 (threshold=0.7 or 0.8). We decided to do this because when we did the explainability, we noticed that a lot of predictions on the target set had high prediction probability (near 100%), which were normally the correctly predicted ones, on the contrary, the incorrect ones, had more often lower probabilities, so we decided to experiment with the probability threshold to improve accuracy. Of course, since our model was overffited, the choice of the threshold was done by trial and error. We discovered some of the predictions that were wrong, by applying brute force, which is explained in the explainability section.

In [ ]:
#missing values target
target=pd.read_csv('task.csv')
ids=target['i']
target=df_value(target.loc[:,'d1':'x9'])

In [ ]:
pip install catboost

In [ ]:
#best model
from catboost import CatBoostClassifier

best_model=GradientBoostingClassifier(learning_rate= 0.1,
 loss= 'exponential',
 min_samples_split= 0.6500000000000001,
 min_samples_leaf= 0.05,
 max_depth= 34,
 max_features= 'sqrt',
 criterion= 'mse',
 subsample= 1.0,
 n_estimators= 291)

In [ ]:
#y_pred = (best_model.predict_proba(target)[:,1] >= 0.3).astype(int) # set threshold as 0.3

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(best_model, final_df, df[Y]).mean()

In [ ]:
# create data frame with predictions
#substitute model
final=pd.concat([ids, pd.DataFrame(best_model.predict(target[X].loc[:,final_df.columns]))], axis=1).rename(columns={0:'y'})

In [ ]:
# prediction with threshold
final=pd.concat([ids, pd.DataFrame((best_model.predict_proba(target)[:,1] >= 0.7).astype(int))], axis=1).rename(columns={0:'y'})

In [ ]:
#create csv
final.to_csv('predictions.csv', index=False)

# Evaluation

To analyse our final model, we created this section so we did not need to search for it in the model section, to see what was the problem with our model. It seems that its problem is, again, overfiting, as it predicts every observation correctly in this training set. Some attempts to decrease overfitting were tried with catboost's internal mechanisms (l2_reg, od_pval and od_wait), but it was not successful because Kaggle's and the training accuracy decreased, which suggests that the two data sets are very similar.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[X], df[Y], test_size=0.33, random_state=1)

# Get predicted probabilities
target_probabilities = best_model.predict_proba(X_test)[:,1]

# Create true and false positive rates
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, target_probabilities)

# plot ROC curve
plt.title("Receiver Operating Characteristic")
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

best_model=CatBoostClassifier(logging_level='Silent', nan_mode='Max',random_strength= 10, min_data_in_leaf= 30, loss_function= 'CrossEntropy', learning_rate= 0.2 , l2_leaf_reg= 0.1, iterations= 1000, depth= 5, border_count= 200, best_model_min_trees= 3).fit(final_df, df_no_na[Y])

X_train, X_test, y_train, y_test = train_test_split(df[X], df[Y], test_size=0.33)

print(best_model.score(X_test, y_test, scoring='f1'))

matrix = confusion_matrix(y_test, best_model.predict(X_test))
dataframe = pd.DataFrame(matrix, index=[0,1], columns=[0,1])
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")
print(plt.show())

In [ ]:
# classification report
from sklearn.metrics import classification_report

print(classification_report(y_test, best_model.predict(X_test), target_names=['0','1']))

# Validation Curve

To reduce overfitting, which we mentioned in the last section, we tried a validation curve, so we could know in which way to optimize the l2_reg, but we ended up seeing that the alteration of a this parameter would not have a positive impact on the dataset prediction, as the accuracy changed very little and it only decreased. So this was of no help in the final prediction. 

In [ ]:
from sklearn.model_selection import validation_curve

# Create range of values for parameter
param_range = np.arange(1, 250, 2)

# Hyperparameter to examine
param_name="l2_leaf_reg"

# Range of hyperparameter's values
param_range = np.arange(0.1, 10, 0.1)

# Calculate accuracy on training and test set using range of parameter values
train_scores, test_scores = validation_curve(best_model, final_df, df[Y], param_name=param_name, param_range=param_range, cv=5, scoring="f1", n_jobs=-1)

# Calculate mean and standard deviation for training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot mean accuracy scores for training and test sets
plt.plot(param_range, train_mean, label="Training score", color="black")
plt.plot(param_range, test_mean, label="Cross-validation score", color="dimgrey")

# Plot accurancy bands for training and test sets
plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, color="gray")
plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, color="gainsboro")

# Create plot
plt.title("Validation Curve With Random Forest")
plt.xlabel("Number Of Trees")
plt.ylabel("Accuracy Score")
plt.tight_layout()
plt.legend(loc="best")

# Explainability

Before starting this part, as suggested in previous ones, we would like to say that the explainability models here were applied to our best model only,ending up helping in the feature selection. We removed x4 from the data set because it had really low explainability, as said. This may mean that it could help in the predictions of this data set but may not be as important in the prediction of other data sets or even skew the predictions. So for the sake of getting a better accuracy we tried to remove it, which made the prediction slightly better. We attempted to do this with d2 and d11, but it only worsen our results.

Another thing that should be said is that our best prediction was accomplished by trial and error. We removed the variables with the lowest explainability in the data set (x4, d11, d2) and then saw which were the predictions that were not compatible between model predictions using Excel. Afterwards we proceeded to substitute these predictions, one by one, by their opposite in our best prediction set, so we could know some of the predictions that were incorrectly and correctly predicted by our model and also increase our accuracy. Even though our highest prediction with a model is 0.97252, our best accuracy on Kaggle is 0.97660 because of this.

We wanted to know which predictions were incorrectly classified, so we could use them in explainability models to see which variables were skewing them, and try to address the problem. However, since we are not experts in explainability, it was not possible to improve our results, because even though, one variables turns a prediction incorrect, it helps many other. 

So, in conclusion, the best model only needed missing value substitution by an arbitrarly big number and feature selection with recursive feature elemination. The further model improvements (thresholdin and further feature selection) were done with the help of the explanation models, primarly the permutation importance table and the lime model.

In [ ]:
pip install lime

In [ ]:
import lime 
from lime import lime_tabular

lime_explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(final_df),
    feature_names=final_df.columns,
    class_names=['notdisc', 'disc'],
    mode='classification'
)

for i in range(0,20):
    lime_exp = lime_explainer.explain_instance(
        data_row=final_df.iloc[i],
        predict_fn=best_model.predict_proba
    )
    lime_exp.show_in_notebook(show_table=True)

In [ ]:
# wrongly and correctly predicted observations in target csv
import lime 
from lime import lime_tabular

target=pd.read_csv('task.csv')
target=df_value(target)
final_target=target[['d2','d11']+numbers[:3]+numbers[4:]]
i_target=target[['i','d2','d11']+numbers[:3]+numbers[4:]]

correct_preds=[23, 221, 429]
wrong_preds=[1085, 1685, 1915,2187, 2685, 2700, 3456, 5056] #1685 was classified as 1, 1085 as 1, 1915 as 0

lime_explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(final_target),
    feature_names=final_target.columns,
    class_names=['notdisc', 'disc'],
    mode='classification'
)

for i in correct_preds+wrong_preds:
    row=i_target.loc[target.loc[:,'i']==i,['d2','d11']+numbers[:3]+numbers[4:]]
    row=row.squeeze() 
    if i in correct_preds:
        print('Correctly classified')
    else:
        print('Wrongly classified')
    lime_exp = lime_explainer.explain_instance(
        data_row=row,
        predict_fn=best_model.predict_proba
    )
    lime_exp.show_in_notebook(show_table=True)

In [ ]:
pip install shap

In [ ]:
import shap
shap.initjs()

shap_explainer = shap.TreeExplainer(best_model)
shap_values = shap_explainer.shap_values(final_df)

shap.summary_plot(shap_values, final_df)

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values[0,:], final_df.iloc[0,:])

In [ ]:
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values[1,:], final_df.iloc[1,:])

In [ ]:
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values[2,:], final_df.iloc[2,:])

In [ ]:
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values[3,:], final_df.iloc[3,:])

In [ ]:
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values[7,:], final_df.iloc[7,:])

In [ ]:
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values[17,:], final_df.iloc[17,:])

Wrongly classified and correctly classified with shap

In [ ]:
target=pd.read_csv('task.csv')
target=df_value(target)
final_target=target[['d2','d11']+numbers[:3]+numbers[4:]]
i_target=target[['i','d2','d11']+numbers[:3]+numbers[4:]]

shap_values_target = shap_explainer.shap_values(final_target)
row=i_target.loc[target.loc[:,'i']==23,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==23,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==221,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==221,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==429,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==429,:], row)

wrongly classified

In [ ]:
row=i_target.loc[target.loc[:,'i']==1085,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==1085,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==1685,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==1685,:], row)

In [ ]:
target=pd.read_csv('task.csv')
target=df_value(target)
final_target=target[['d2','d11']+numbers[:3]+numbers[4:]]
i_target=target[['i','d2','d11']+numbers[:3]+numbers[4:]]

row=i_target.loc[target.loc[:,'i']==1915,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==1915,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==2187,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==2187,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==2685,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==2685,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==2700,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==2700,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==3456,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==3456,:], row)

In [ ]:
row=i_target.loc[target.loc[:,'i']==5056,['d2','d11']+numbers[:3]+numbers[4:]]
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target[target.loc[:,'i']==5056,:], row)

In [ ]:
# create a SHAP dependence plot to show the effect of a single feature across the whole dataset
shap.initjs()
for i in final_df.columns:    
    shap.dependence_plot(i, shap_values, final_df)

In [ ]:
# visualize the training set predictions
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values, final_df)

In [ ]:
# visualize the target set predictions
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_target, final_target)

In [ ]:
pip install eli5

In [ ]:
#permutation importance
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(best_model, random_state=1).fit(final_df, df[Y])
eli5.show_weights(perm, feature_names = final_df.columns.tolist())

In [ ]:
pip install pdpbox

In [ ]:
# Partial Plots
#perguntar pelo que substituir o x9
from pdpbox import pdp, get_dataset, info_plots

for i in final_df.columns: 
    feature_to_plot = i
    pdp_dist = pdp.pdp_isolate(model=best_model, dataset=final_df, model_features=final_df.columns, feature=feature_to_plot)

    pdp.pdp_plot(pdp_dist, feature_to_plot)
    plt.show()